In [46]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import math
import matplotlib
import matplotlib.pyplot as plt
import random

from src.tf_tools.rnn_regression import coolRNN
from src.tf_tools.cnn_regression import smallerResNet
from src.grammar.sentences import sentences_Cspacer

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# 3.10.4

In [47]:
def one_hot_seqs(seqs) -> np.array:
    static_1hotmap = {
        'A' : np.array([1,0,0,0]),
        'a' : np.array([1,0,0,0]),
        'C' : np.array([0,1,0,0]),
        'c' : np.array([0,1,0,0]),
        'G' : np.array([0,0,1,0]),
        'g' : np.array([0,0,1,0]),
        'T' : np.array([0,0,0,1]),
        't' : np.array([0,0,0,1]),
    }
    onehot_seqs = []
    for seq in seqs:
        onehot_seqs.append(
            [static_1hotmap[seq[i]] if seq[i] in static_1hotmap.keys() else static_1hotmap[random.choice(['A','C','G','T'])] for i in range(len(seq))]
        )
    return np.stack(onehot_seqs)

In [ ]:
activity_df = pd.read_csv('Data/activity.csv', index_col=0)
activity_df = activity_df.set_index('label')
activity_df['sentence'] = sentences_Cspacer(activity_df.index.to_list(), "Data/Motifs/summarized_motifs.parquet")

In [17]:
FEATURE_KEY = 'sequence'
LABEL_KEY = 'expression_log2'

In [32]:
sentence_df = pd.read_csv("Data/Sentences/sentences.csv", index_col=0)


In [37]:
max([len(s.split()) for s in test_df['sentence']])

36

In [21]:

vocab = set()

for sentence in sentence_df['sentence'].values:
    vocab.update(sentence.split())
    
vocab_size = len(vocab)+1

In [19]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences)

In [41]:
sentence_df['tokenized'] = tokenize(sentence_df['sentence'].values)
train_df = sentence_df[sentence_df['label'].isin(train_i)]
validation_df = sentence_df[sentence_df['label'].isin(validation_i)]
test_df = sentence_df[sentence_df['label'].isin(test_i)]

In [42]:
max([len(s) for s in sentence_df['tokenized'].values])

81

In [38]:
train_tokenized = tokenize(train_df['sentence'].to_list())
validation_tokenized = tokenize(validation_df['sentence'].to_list())
test_tokenized = tokenize(test_df['sentence'].to_list())

In [39]:
max_sentence_length = max([
    max([len(s) for s in train_tokenized]),
    max([len(s) for s in validation_tokenized]),
    max([len(s) for s in test_tokenized])
])
input_shape = (max_sentence_length,1)
input_shape

(81, 1)

In [59]:
x_train = pad_sequences(train_tokenized, max_sentence_length, padding='post')
x_train = x_train.reshape(*x_train.shape,1)
y_train = train_df['expression_log2'].values

x_validation = pad_sequences(validation_tokenized, max_sentence_length, padding='post')
x_validation = x_validation.reshape(*x_validation.shape,1)
y_validation = validation_df['expression_log2'].values

x_test = pad_sequences(test_tokenized, max_sentence_length, padding='post')
x_test = x_test.reshape(*x_test.shape,1)
y_test = test_df['expression_log2'].values


In [22]:
def coolRNN(input_shape=(input_shape)):
    X_input = keras.Input(input_shape, name='InputLayer')

    X = keras.layers.LSTM(64,return_sequences=True)(X_input)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(1)(X)

    model = keras.Model(inputs=X_input, outputs=X, name='coolRNN')
    
    return model

In [23]:

model = coolRNN()
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "coolRNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 33, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 33, 64)            16896     
                                                                 
 flatten_3 (Flatten)         (None, 2112)              0         
                                                                 
 dense_9 (Dense)             (None, 1)                 2113      
                                                                 
Total params: 19009 (74.25 KB)
Trainable params: 19009 (74.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
history = model.fit(x_train,y_train,batch_size=128,epochs=20)

Epoch 1/20


ValueError: in user code:

    File "/ref/bclab/software/spack-0.17.2/var/spack/environments/tensorflow-gpu/.spack-env/._view/gctfpgbf2atvptgnwhwmsyrtv7diigjw/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/ref/bclab/software/spack-0.17.2/var/spack/environments/tensorflow-gpu/.spack-env/._view/gctfpgbf2atvptgnwhwmsyrtv7diigjw/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/ref/bclab/software/spack-0.17.2/var/spack/environments/tensorflow-gpu/.spack-env/._view/gctfpgbf2atvptgnwhwmsyrtv7diigjw/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/ref/bclab/software/spack-0.17.2/var/spack/environments/tensorflow-gpu/.spack-env/._view/gctfpgbf2atvptgnwhwmsyrtv7diigjw/lib/python3.10/s

In [70]:
history.history

{'loss': [10.18856143951416,
  9.812065124511719,
  9.742993354797363,
  9.688549995422363,
  9.623767852783203,
  9.544726371765137,
  9.47740650177002,
  9.3971529006958,
  9.326369285583496,
  9.242609024047852,
  9.166191101074219,
  9.083224296569824,
  8.992240905761719,
  8.901432991027832,
  8.835554122924805,
  8.741174697875977,
  8.670454978942871,
  8.584476470947266,
  8.509294509887695,
  8.434361457824707]}

In [72]:
test = smallerResNet()
test.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 164, 4)]             0         []                            
                                                                                                  
 zero_padding1d (ZeroPaddin  (None, 170, 4)               0         ['input_1[0][0]']             
 g1D)                                                                                             
                                                                                                  
 conv1 (Conv1D)              (None, 82, 64)               1856      ['zero_padding1d[0][0]']      
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, 82, 64)               256       ['conv1[0][0]']        